In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
credible = pd.read_csv('corpus/credible.csv', header=None, names=['id', 'type', 'domain', 'content'])

In [ ]:
credible['domain'].unique()

In [ ]:
credible = credible[(credible['domain'] != 'www.msn.com') & (credible['domain'] != 'feed.reuters.com')]

In [ ]:
others = credible[~credible['domain'].isin(['nytimes.com', 'nationalreview.com', 'www.reuters.com', 'weeklystandard.com'])]

In [ ]:
len(others)

In [2]:
import spacy
from spacy_langdetect import LanguageDetector
nlp = spacy.load('en')
nlp.add_pipe(LanguageDetector(), name="language_detector", last=True)
import re
from unidecode import unidecode

In [3]:
class Cleaner(dict):
    """ Multiple-string-substitution dict """
    def _make_regex(self):
        """ Build re object based on the keys of the dictionary it is instantiated with"""
        return re.compile("|".join(map(re.escape, self.keys(  ))))

    def __call__(self, match):
        """ Handler invoked for each regex match """
        return self[match.group(0)]

    def clean(self, text):
        """ Substitutes with value for each key and returns the modified text. """
        return self._make_regex(  ).sub(self, text)

In [441]:
replacements = {#"\n": " ", # new line characters
                "\t": " ", # tabs
                "-": " ",
                "won't": "will not",
                "can't": "can not",
                "&": " and ",
                "$$": "$",
                "Loading...": " ",
                "Continued...": " ",
                "\N{COPYRIGHT SIGN}": " ",
                "\N{NO-BREAK SPACE}": " ",
                "\N{LEFT-POINTING DOUBLE ANGLE QUOTATION MARK}": " ",
                "\N{RIGHT-POINTING DOUBLE ANGLE QUOTATION MARK}": " ",
                '."': '".',
                '?"': '"?',
                '!"': '"!',
                
               }

In [453]:
entities = {'PERSON': 'person',
            'FAC': 'landmark',
            'ORG': 'organization',
            'GPE': 'place',
            'LOC': 'location',
            'EVENT': 'event',
            'WORK_OF_ART': 'artwork',
            'LAW': 'law',
            'DATE': 'date',
            'TIME': 'time',
            'PERCENT': 'percent',
            'MONEY': 'money',
            'QUANTITY': 'quantity',
            'CARDINAL': 'number'
}

ent_order = {'PERSON': 8,
            'FAC': 2,
            'ORG': 1,
            'GPE': 6,
            'LOC': 7,
            'EVENT': 3,
            'WORK_OF_ART': 5,
            'LAW': 4,
            'DATE': 9,
            'TIME': 10,
            'PERCENT': 12,
            'MONEY': 11,
            'QUANTITY': 13,
            'CARDINAL': 14,
}

drop_ents = ['NORP', 'PRODUCT', 'LANGUAGE','ORDINAL']

In [454]:
preprocess = Cleaner(replacements)

In [487]:
def process(in_doc):
    count = 0
    out_doc = ""
    doc = re.sub(r'\n[\n ]*', ' ', in_doc)
    doc = nlp(doc)
    if doc._.language['language'] != 'en':
        return np.nan
    for sent in doc.sents:
        ending = sent[-1]
        if ending.pos_ != 'PUNCT':
            continue
        out_doc += (sent.text + ' ')
        if ending.text in ['.', '?', '!']:
            count += 1
    if count < 13:
        return count
    print(count)
    ents = [ent for ent in doc.ents if ent.label_ not in drop_ents]
    ents = sorted(ents, key=lambda ent: ent_order[ent.label_])
    converted = set([])
    for ent in ents:
        if (ent.text, ent.label_) in converted:
            continue
        converted.add((ent.text, ent.label_))
        pattern = r'\b{}\b'.format(ent.text)
        out_doc = re.sub(pattern, entities.get(ent.label_, ent.text), out_doc)
    return out_doc

In [498]:
def convert_quotes(qq):
    num = 0
    if qq[-2] in ['.', '?', '!']:
        punct = qq[-2]
    else:
        punct = ''
    length = len(qq.split())
    if length <= 4:
        num = 1
    elif length <= 12:
        num = 2
    elif length <= 25:
        num = 3
    else:
        num = 4
    return 'quote ' * num + punct

def reformat(article):
    text = unidecode(article)
    if text.count('\N{QUOTATION MARK}') % 2 != 0:
        return np.nan
    text = preprocess.clean(text)
    text = re.sub(r'^(.{0,50})\(\w+\)', ' ', text) # delete dateline
    text = re.sub(r'\S*@\S+', 'email', text) # replace email address or Twitter handle with "email"
    text = re.sub(r'[-a-zA-Z0-9@:%_\+.~#?&\/=]{2,256}\.[a-z]{2,4}(\/[-a-zA-Z0-9@:%_\+.~#?&\/=]*)?', ' website',
                  text) # URLs
    text = re.sub('[\[\(][^\[\(]*[\]\)]', '', text) # delete text inside parentheses or brackets
    text = re.sub(r"\b(\w*)n't", lambda m: m.group(1) + ' not', text) # replace "xxn't" contractions with "xx not"; "won't" already handled
    text = re.sub(r'("[^"]*")', lambda m: convert_quotes(m.group(1)), text) # replace quoted text
    text = re.sub(r"^'|'$|(?<= )'|(?<!s)'(?= )", '\1"', text) # replace single quotes, but not apostrophes, with double quotes
    text = re.sub(r'("[^"]*")', lambda m: convert_quotes(m.group(1)), text) # replace quoted text    
    if text.count('\N{QUOTATION MARK}') % 2 != 0:
        return np.nan
    text = re.sub(r'("[^"]*")', lambda m: convert_quotes(m.group(1)), text) # replace quoted text
    text = re.sub(r'(?i)please share this.*', '', text)
    text = re.sub(' +', ' ', text) # reduce all multiple spaces to single spaces
    return process(text)

In [ ]:
others.sample(10)

In [624]:
fake = pd.read_csv('corpus/fake.csv', header=None, names=['id', 'type', 'domain', 'content'], dtype={'id': str})

In [ ]:
fake[fake['domain'] != 'beforeitsnews.com'] .sample(10)

In [9]:
bias = pd.read_csv('corpus/bias.csv', header=None, names=['id', 'type', 'domain', 'content'], dtype={'id': str})

In [10]:
def show_articles(df):
    for ix, row in df.sample(10).iterrows():
        yield helper(ix, row)
        
def helper(ix, row):
    print(ix, ', ', row['domain'])
    print(reformat(row['content']))

In [ ]:
biases = show_articles(bias)

In [ ]:
next(biases)

In [623]:
fake['domain'].unique()

NameError: name 'fake' is not defined

In [11]:
def show_contents(df):
    for domain in df['domain'].unique():
        print(domain)
        num = min(len(df[df['domain'] == domain]), 10)
        yield df[df['domain'] == domain].sample(num)

In [103]:
bias_domains = show_contents(bias)

In [620]:
next(bias_domains)

StopIteration: 

In [618]:
bias.loc[1083375, 'content']

'Ellen DeGeneres and Celine Dion burning a U.S. flag in front of a …\n\nChuck Norris has a reputation as being so tough, it’s legendary. However, it …\n\nIn the wake of the horrific mass shooting at a small church in …\n\nLess than one week after suspected mass shooter Devin Kelley gunned down at …\n\nSometimes even the greatest friendships fail, and duos like Sonny and Cher find …\n\nAnother celebrity ego-stroking awards show took place Wednesday night and like all other …\n\nA New York church is warning potential shooters not to mess with the …\n\nA British man sentenced to life behind bars earlier this year for beating …\n\nWith the recent release of recording artist, Taylor Swift’s latest single and video, …\n\nIn the wake of any mass shooting, hysteria erupts about guns. All of …\n\nAd Blocker Detected\n\nOur website is made possible by displaying online advertisements to our visitors. Please consider supporting us by disabling your ad blocker.'

In [621]:
fake_keeper = []

In [622]:
bias_keepers = ['wnd.com', 'frontpagemag.com', 'americanthinker.com', 'dailywire.com', 'thegatewaypundit.com', 
               'antiwar.com', 'truthrevolt.org', 'patriotpost.us', 'russia-insider.com', 'paulcraigroberts.org',
               'vdare.com', 'off-guardian.org', 'jamesrgrangerjr.com', 'americablog.com', 'americasfreedomfighters.com',
               'heartland.org', 'palmerreport.com', 'thefederalistpapers.org', 'conservativetribune.com',
               'winningdemocrats.com', '100percentfedup.com', 'cowgernation.com', 'usherald.com', 'darkpolitricks.com',
               'newslogue.com', 'usapoliticstoday.com', 'counterjihad.com', 'platosguns.com', 'meanlefthook.com',
               'americanpatriotdaily.com', 'endingthefed.com', 'conservativefiringline.com', 'politicalcult.com',
               'readconservatives.news']

In [608]:
reformat(bias.loc[543267, 'content'])

nan

In [420]:
for ent in nlp(bias.loc[265413, 'content']).ents:
    print(ent.text, '|', ent.label_)



 | DATE


Germany | PERSON
Berlin | GPE
Turkey | GPE
NATO | ORG
Turkish | NORP
Soner Polat | PERSON
Sputnik | PERSON
Earlier this week | DATE
Turkish | NORP
Peter Steudtner | PERSON
German | NORP
six | CARDINAL
Amnesty International’s | ORG
Turkey | GPE
Idil Eser | PERSON
Turkish | NORP
July 5 | DATE
German | NORP
Sigmar Gabriel | PERSON
Berlin | GPE
Ankara | GPE
Turkey | GPE
Germany | GPE
Turkey Spiral | ORG
Crisis https://t.co/qDS58JclNa — Janet Orendorff | FAC
July 22, 2017 | DATE
Turkish | NORP
thousands | CARDINAL
July 15, 2016 | DATE
Islamic | NORP
Fethullah Gulen | PERSON
Ankara | GPE
Germany | GPE
Turkey | GPE
MFA | ORG
— Srbija Evropa | PERSON
July 20, 2017

 | DATE
Sputnik Turkey | PERSON
Retired Rear Admiral | PERSON
the Turkish Armed Forces Soner Polat | ORG
Germany | GPE
Turkey | GPE
NATO | ORG
Turkey | GPE
Eurasia | GPE
Turkey | GPE
NATO | ORG
1952 | DATE
Germany | GPE
Turkey | GPE
NATO | ORG
Polat | PERSON
Turkey | GPE
Turkey | GPE
July 15 last year | DATE
Turkey | GPE

In [450]:
tester = set([('Hillary', 'person'), ('NAACP', 'org'), ('Whitman', 'bridge'), ('Hillary', 'person')])
tester

{('Hillary', 'person'), ('NAACP', 'org'), ('Whitman', 'bridge')}

In [ ]:
def silliness(string):
    new_string = re.sub(r"^'|'$|(?<= )'|'(?= )", '"', string)
    print(new_string.count('\N{QUOTATION MARK}'))
    if new_string.count('\N{QUOTATION MARK}') % 2 != 0:
        return np.nan
    return 'WTF'

In [ ]:
tester = nlp('Victims seek to resume Marcos from Heroes\' Cemetery')

In [ ]:
for token in tester:
    print(token.pos_)

In [ ]:
import regex

In [ ]:
trial = 'How about \u00a9 for a change'
print(trial)
trial2 = re.sub('\N{COPYRIGHT SIGN}', 'this', trial)
print(trial2)

In [ ]:
'How about \u00a9 for a change'.replace('\N{COPYRIGHT SIGN}', 'this')

In [ ]:
print('\N{RIGHT-POINTING DOUBLE ANGLE QUOTATION MARK}')

In [ ]:
import os
scraper_data = os.listdir('./data')
scraped = pd.DataFrame()
for file in scraper_data:
    try:
        df = pd.read_json('./data/{}'.format(file))
        scraped = pd.concat([scraped, df])
    except:
        pass

In [ ]:
len(scraped)

In [ ]:
scraped.head()

In [ ]:
scraped = scraped.drop_duplicates(['id'], keep='last')
len(scraped)

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
scraped.head()

In [ ]:
len(df1.dropna())

In [ ]:
len(df1.drop('id', axis=1).dropna())

In [ ]:
len(df1.drop('domain', axis=1).dropna())

In [ ]:
df1 = df1.dropna()

In [ ]:
df1['type'].unique()

In [ ]:
len(df1[(df1['type'] != 'unreliable') & (df1['type'] !='unknown')])

In [ ]:
df1 = df1[(df1['type'] != 'unreliable') & (df1['type'] !='unknown')]

In [ ]:
df1.groupby('type').size()

In [ ]:
df1[df1['type'] == 'political']['domain'].unique()

In [ ]:
media_bias = pd.read_csv('data/media_bias.csv')

In [ ]:
media_bias[media_bias['Vertical Rank'] >= 40]

In [ ]:
import scrapy
import re
from scrapy.crawler import CrawlerProcess

In [ ]:
some_data = pd.read_json('data/abc_20181207.json')

In [ ]:
some_data.loc[13, 'article']

In [ ]:
len(some_data)

In [ ]:
"https://cbsnews.com/world".count("/")